# Lesson 3: Vectorstores and embeddings

# Vectorstore ingestion

In [1]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [2]:
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings();

await embeddings.embedQuery("This is some sample text");

[
   -0.010416139,   0.0024119338, -0.00073926017,  -0.010882434,  -0.0114866495,
    0.022907624,   -0.014645643,    0.001761746,   -0.01757477,     -0.0192692,
    0.005201502,    0.034151275,   -0.012268188,  0.0019308605,    0.004745057,
    0.013174511,    0.024588916,   0.0017256244,  0.0045020576,  -0.0062358915,
  -0.0050931373,   -0.000690414,   -0.008183171,   0.014002022,   -0.009010682,
  -0.0040620314, -0.00077127694,   -0.019728929,   0.003966802,  -0.0016689793,
    0.016051099,   -0.021935627, -0.00076758274,  -0.022316543,   0.0063278372,
    0.007046984,   -0.011184542,   -0.013502888,   0.028818421,   -0.023577515,
    0.006922201,    0.017561637,    0.017298933,  -0.005887811,  -0.0038847073,
    -0.01435667,  -0.0017584623,  -0.0060027433,  -0.007894198,    0.010448976,
    0.032837763, -0.00082504883,   -0.024063513, -0.0033363167, -0.00088251487,
   0.0026319467,   -0.022487301,    0.025271943,   0.019781468,    0.010797056,
   -0.011677109,  -0.0025071632,   0.0

In [3]:
import { similarity } from "ml-distance";

const vector1 = await embeddings.embedQuery(
    "What are vectors useful for in machine learning?"
);
const unrelatedVector = await embeddings.embedQuery(
    "A group of parrots is called a pandemonium."
);

In [4]:
similarity.cosine(vector1, unrelatedVector);

0.6959591747618556

In [5]:
const similarVector = await embeddings.embedQuery(
    "Vectors are representations of information."
);

similarity.cosine(vector1, similarVector);

0.8586349152599215

In [6]:
// Peer dependency
import * as parse from "pdf-parse";
import { PDFLoader } from "langchain/document_loaders/fs/pdf";
import { 
    RecursiveCharacterTextSplitter
} from "langchain/text_splitter";

const loader = new PDFLoader("./data/MachineLearning-Lecture01.pdf");

const rawCS229Docs = await loader.load();

const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 128,
  chunkOverlap: 0,
});

const splitDocs = await splitter.splitDocuments(rawCS229Docs);

In [7]:
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const vectorstore = new MemoryVectorStore(embeddings);

In [8]:
await vectorstore.addDocuments(splitDocs);

In [9]:
const retrievedDocs = await vectorstore.similaritySearch(
    "What is deep learning?", 
    4
);

const pageContents = retrievedDocs.map(doc => doc.pageContent);

pageContents

[
  "piece of research in machine learning, okay?",
  "are using a learning algorithm, perhaps without even being aware of it.",
  "some of my own excitement about machine learning to you.",
  "of the class, and then we'll start to talk a bit about machine learning."
]

# Retrievers

In [10]:
const retriever = vectorstore.asRetriever();

In [11]:
await retriever.invoke("What is deep learning?")

[
  Document {
    pageContent: "piece of research in machine learning, okay?",
    metadata: {
      source: "./data/MachineLearning-Lecture01.pdf",
      pdf: {
        version: "1.10.100",
        info: {
          PDFFormatVersion: "1.4",
          IsAcroFormPresent: false,
          IsXFAPresent: false,
          Title: "",
          Author: "",
          Creator: "PScript5.dll Version 5.2.2",
          Producer: "Acrobat Distiller 8.1.0 (Windows)",
          CreationDate: "D:20080711112523-07'00'",
          ModDate: "D:20080711112523-07'00'"
        },
        metadata: Metadata { _metadata: [Object: null prototype] },
        totalPages: 22
      },
      loc: { pageNumber: 8, lines: { from: 2, to: 2 } }
    }
  },
  Document {
    pageContent: "are using a learning algorithm, perhaps without even being aware of it.",
    metadata: {
      source: "./data/MachineLearning-Lecture01.pdf",
      pdf: {
        version: "1.10.100",
        info: {
          PDFFormatVersion: "1.4",